In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [ ]:
from liqpy import LiqPay

liqpay = LiqPay()
liqpay

In [ ]:
from json import dumps
from uuid import uuid4

order_id = str(uuid4())

params = dict(
    action="pay",
    amount=1,
    currency="USD",
    description="Lorem Ipsum",
    order_id=order_id,
)

sep, end = "\n", "\n\n"

print("params:", dumps(params, indent=4), sep=sep, end=end)

data, signature = liqpay.encode(**params)

print("data:", data, sep=sep, end=end)
print("signature:", signature, sep=sep)

In [ ]:
from pprint import pprint
from liqpy.exceptions import LiqPayException

try:
    liqpay.request(**params)
except LiqPayException as e:
    pprint(e.data)
    raise e

In [ ]:
liqpay.checkout(**params)

In [ ]:
liqpay.status(order_id)